# Command Line tools

> Define some wrappers for command line tools for training the models

In [ ]:
#| default_exp cli 

In [ ]:
#| export 

import argparse
from mdetect.core import *
from pathlib import Path
import glob
import numpy as np


from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,  accuracy_score,roc_auc_score
from sklearn.model_selection import cross_val_score

from sklearn.metrics import RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.calibration import CalibrationDisplay
from sklearn.manifold import TSNE
from imblearn.over_sampling import SMOTENC 
from imblearn.pipeline import Pipeline as imbPipeline

import pandas as pd
import joblib

In [ ]:
#| export 

def main_train():
    
    parser = argparse.ArgumentParser(description='Train a model')
    parser.add_argument('--data', type=str, default='data', help='data directory')
    parser.add_argument('--save', type=str, default='inference.pkl', help='Path to save the model')
    args = parser.parse_args()
    
    
    # Get the filename of the file that starts with X_* in the data directory
    features = glob.glob(f'{args.data}/X*')[0]
    labels = glob.glob(f'{args.data}/y*')[0]
    
    if len(features) == 0 or len(labels) == 0:
        raise Exception("No data found")
    if len(features) > 1 or len(labels) > 1:
        raise Exception(f"Multiple data files found {features} {labels}")
    
    X = pd.read_csv(features[0], delimiter=',')
    y = pd.read_csv(labels, delimiter=',').to_numpy() 
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
    
    rf_pipe = imbPipeline([('resample', SMOTENC(categorical_features=[8,9,10])), 
                       ('model', RandomForestClassifier(n_estimators = 30, random_state=42)),
                       ])
    modelcand = ModelCandidate(rf_pipe, 'RandomForest')
    fit_model = evaluate(modelcand, X_train, y_train, X_test, y_test)
    # print all the model metrics
    print(f"Accuracy: {fit_model.accuracy_score}")
    print(f"Confusion Matrix: {fit_model.confusion_matrix}")
    print(f"F1 Score: {fit_model.f1_score}")
    print(f"AUC: {fit_model.auc_score}")
    print(f"Cross validation score: {fit_model.cv_scores}")
    
    # Save the model 
    pipeline_filename = args.save
    joblib.dump(fit_model.modelcand, pipeline_filename)
    
    print(f"Model saved to {pipeline_filename}")


In [ ]:
#| export 

def main_transform():
    
    parser = argparse.ArgumentParser(description='Transform pcap files to features')
    parser.add_argument('--malware -md', type=str, default='data', help='path to malware pcap files')
    parser.add_argument('--benign -bd', type=str, default='data', help='path to benign pcap files')
    parser.add_argument('--save -s', type=str, default=".", help='save the features to disk at this path')
    args = parser.parse_args()
    
    X, y = load_training_validation(Path(args.malware), Path(args.benign), save=False, load=False)
    
    savepath = Path(args.save)
    benign_filehash = hash_datafiles(benign_files)
    malware_filehash = hash_datafiles(malware_files)
    
    X_filename = f'X_{benign_filehash}_{malware_filehash}.csv.gz'
    y_filename = f'y_{benign_filehash}_{malware_filehash}.csv.gz'
    
    X.to_csv(savepath / X_filename, index=False)
    pd.DataFrame(y).to_csv(savepath / y_filename, index=False)
    
    print(f"Saved X to {savepath / X_filename}")
    print(f"Saved y to {savepath / y_filename}")
    
    

In [ ]:
def main_predict():
    
    parser = argparse.ArgumentParser(description='Predict if a pcap file is malware or benign')
    parser.add_argument('--data -d', type=str, default='data', help='path to pcap file')
    parser.add_argument('--model -m', type=str, help='path to model in pickle format')
    
    args = parser.parse_args()
    pipeline = joblib.load(args.model)
    X = collect_flow_stats(args.data)
    y_pred = pipeline.predict(args.data)
    
    print(f"Prediction: {y_pred.sum} malware flows detected out of {len(y_pred)} flows")
    print("Give a probability of the pcap containing malware")
    
    
    
    
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()